In [95]:
def colocar_tipo_de_carro_de_acordo_com_modelo(DataFrame):
    dict_modelos = df.groupby('model')['brand'].apply(list).to_dict()
    for key in dict_modelos:
        dict_modelos[key] = list(dict.fromkeys(dict_modelos[key]))
        dict_modelos[key] = dict_modelos[key][0]
    DataFrame.vehicleType = DataFrame.vehicleType.fillna(df.model.map(dict_modelos))
    return(DataFrame)
    
def colocar_marca_do_carro_de_acordo_com_modelo(DataFrame):
    dict_modelos = df.groupby('model')['brand'].apply(list).to_dict()
    for key in dict_modelos:
        dict_modelos[key] = list(dict.fromkeys(dict_modelos[key]))
        dict_modelos[key] = dict_modelos[key][0]
    DataFrame.brand = DataFrame.brand.fillna(df.model.map(dict_modelos))
    return(DataFrame)

In [96]:
import pandas as pd
import matplotlib.pyplot as plt
file_path='C:/Users/eduar/OneDrive/Área de Trabalho/data_science_test/autos.csv'

df = pd.read_csv(file_path, encoding = 'ISO-8859-1')
df['notRepairedDamage'].fillna('ja', inplace = True)

colocar_marca_do_carro_de_acordo_com_modelo(df)
colocar_tipo_de_carro_de_acordo_com_modelo(df)

features = ['offerType','yearOfRegistration','powerPS','model','kilometer','gearbox','fuelType','notRepairedDamage']
X = df[features]
y = df.price

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train.shape
y_train.shape


#print(df.isnull().sum())
#df.info
#df.describe()
#df.columns
#df.isnull()

(297222,)

In [97]:
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

In [99]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_test = X_test.select_dtypes(exclude=['object'])


OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[object_cols]))

OH_cols_train.index = X_train.index
OH_cols_test.index = X_test.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

label_X_train = X_train.copy()
label_X_test = X_test.copy()
label_encoder = LabelEncoder()

for col in object_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_test[col] = label_encoder.transform(X_test[col])


forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(label_X_train, y_train)
print(mean_absolute_error(y_test, predictions))

[ 1828.27       18279.49008333  6139.28797619 ...  5518.6855
   240.28183333  1707.32047619]
46793.51013313737
